In [322]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from matplotlib.colors import ListedColormap
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn import svm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Initially importing the datasets

In [323]:
cpy_tweet = pd.read_csv('dataset/Company_Tweet.csv')
tweet = pd.read_csv('dataset/Tweet.csv')
company = pd.read_csv('dataset/Company.csv')

Exploring the datasets

In [324]:
company

,ticker_symbol,company_name
0,AAPL,apple
1,GOOG,Google Inc
2,GOOGL,Google Inc
3,AMZN,Amazon.com
4,TSLA,Tesla Inc
5,MSFT,Microsoft


In [325]:
tweet

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,1420070510,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0
3,550442977802207232,ShowDreamCar,1420070807,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1
4,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
...,...,...,...,...,...,...,...
3717959,1212159765914079234,TEEELAZER,1577836383,That $SPY $SPX puuump in the last hour was the...,1,0,6
3717960,1212159838882533376,ShortingIsFun,1577836401,In 2020 I may start Tweeting out positive news...,0,0,1
3717961,1212160015332728833,Commuternyc,1577836443,Patiently Waiting for the no twitter sitter tw...,0,0,5
3717962,1212160410692046849,MoriaCrypto,1577836537,I don't discriminate. I own both $aapl and $ms...,1,0,1


In [354]:
tweet.isna().sum()

tweet_id           0
writer         47273
post_date          0
body               0
comment_num        0
retweet_num        0
like_num           0
dtype: int64

We can see that there are no null values except in writer column. We'll not be making any changes to that column as it doesn't affect our analysis

In [326]:
cpy_tweet

,tweet_id,ticker_symbol
0,550803612197457920,AAPL
1,550803610825928706,AAPL
2,550803225113157632,AAPL
3,550802957370159104,AAPL
4,550802855129382912,AAPL
...,...,...
4336440,1212158772015034369,TSLA
4336441,1212159099632267268,TSLA
4336442,1212159184931717120,TSLA
4336443,1212159838882533376,TSLA


Merging the company tweets and tweets dataframes based on tweet id

In [327]:
tweets = pd.merge(tweet,cpy_tweet,on='tweet_id',how='inner')
tweets.head()

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,AAPL
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0,AAPL
2,550441732014223360,DozenStocks,1420070510,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0,AMZN
3,550442977802207232,ShowDreamCar,1420070807,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1,TSLA
4,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL


converting the postdate to datetime format

In [328]:
tweets['post_date'] = pd.to_datetime(tweets['post_date'], unit='s')
tweets.tail()

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol
4336440,1212159838882533376,ShortingIsFun,2019-12-31 23:53:21,In 2020 I may start Tweeting out positive news...,0,0,1,TSLA
4336441,1212160015332728833,Commuternyc,2019-12-31 23:54:03,Patiently Waiting for the no twitter sitter tw...,0,0,5,TSLA
4336442,1212160410692046849,MoriaCrypto,2019-12-31 23:55:37,I don't discriminate. I own both $aapl and $ms...,1,0,1,AAPL
4336443,1212160410692046849,MoriaCrypto,2019-12-31 23:55:37,I don't discriminate. I own both $aapl and $ms...,1,0,1,MSFT
4336444,1212160477159206912,treabase,2019-12-31 23:55:53,"$AAPL #patent 10,522,475 Vertical interconnect...",0,0,0,AAPL


Creating a separate dataframe for each company using their ticker symbol

In [329]:
tweet_amazon = tweets[tweets['ticker_symbol']=='AMZN']
tweet_apple = tweets[tweets['ticker_symbol']=='AAPL']
tweet_tesla = tweets[tweets['ticker_symbol']=='TSLA']
tweet_microsoft = tweets[tweets['ticker_symbol']=='MSFT']
tweet_google = tweets[(tweets['ticker_symbol']=='GOOG') | (tweets['ticker_symbol']=='GOOGL')]


Extracting tweets based on timeline. Initially, we tried to use 3 years of data but the sentimental analysis took a long time to run. Hence, after trying multiple time periods, we have decided to use the last 6 months in the dataset

In [330]:
def time_mask(df):
    mask = (df['post_date'] > '2019-7-1') & (df['post_date'] <= '2019-12-31')
    df = df.loc[mask]
    return df
    

tweet_amazon = time_mask(tweet_amazon)
tweet_apple = time_mask(tweet_apple)
tweet_google = time_mask(tweet_google)
tweet_microsoft = time_mask(tweet_microsoft)
tweet_tesla = time_mask(tweet_tesla)

Changing post_date column to date column

In [331]:
tweet_amazon.rename(columns={'post_date': 'Date'}, inplace=True)
tweet_apple.rename(columns={'post_date': 'Date'}, inplace=True)
tweet_google.rename(columns={'post_date': 'Date'}, inplace=True)
tweet_microsoft.rename(columns={'post_date': 'Date'}, inplace=True)
tweet_tesla.rename(columns={'post_date': 'Date'}, inplace=True)

tweet_amazon['Date'] = pd.to_datetime(tweet_amazon['Date']).dt.date
tweet_apple['Date'] = pd.to_datetime(tweet_apple['Date']).dt.date
tweet_google['Date'] = pd.to_datetime(tweet_google['Date']).dt.date
tweet_microsoft['Date'] = pd.to_datetime(tweet_microsoft['Date']).dt.date
tweet_tesla['Date'] = pd.to_datetime(tweet_tesla['Date']).dt.date

In [332]:
tweet_amazon.to_csv('dataset/EDA_amazon_tweets.csv')
tweet_apple.to_csv('dataset/EDA_tesla_tweets.csv')
tweet_google.to_csv('dataset/EDA_microsoft_tweets.csv')
tweet_microsoft.to_csv('dataset/EDA_google_tweets.csv')
tweet_tesla.to_csv('dataset/EDA_apple_tweets.csv')

In [355]:
tweet_amazon.head(20)

,tweet_id,writer,Date,body,comment_num,retweet_num,like_num,ticker_symbol
3935144,1145483574717497344,muckdog,2019-07-01,Had an interesting conversation with friends y...,0,1,0,AMZN
3935153,1145485860021792768,EliteOptions2,2019-07-01,$NVDA - Trade Idea - July 5th 170C - bid/ask: ...,2,2,14,AMZN
3935162,1145487616004874241,EliteOptions2,2019-07-01,$BABA - Trade Idea - July 5th 172.50C - bid/as...,1,3,18,AMZN
3935169,1145488552710430720,EliteOptions2,2019-07-01,Two Day Risk-Free Trial Join us tonight (9PM E...,1,1,1,AMZN
3935178,1145489717045207041,ExactOptionPick,2019-07-01,ACTIVE TRADERS Try one of these FREE trading g...,0,0,0,AMZN
3935185,1145490210001752065,Trading_Analyst,2019-07-01,[WATCH] Thoughts On The Market To Help You Mak...,0,1,0,AMZN
3935192,1145492061405081601,BoogDmc,2019-07-01,$AMZN- position,0,1,0,AMZN
3935198,1145494084393586695,mikeo188,2019-07-01,"$ES $ES_F $SPX $3,000 or else the 4th of July ...",0,2,3,AMZN
3935206,1145496736556826624,luke_sosnowski,2019-07-01,Market traders give Revolution Swing from #TRI...,1,3,11,AMZN
3935216,1145501346835193856,DentonBlackwell,2019-07-01,$AMZN eat a dick shorts,0,1,1,AMZN


While exploring the tweets, we can find many tweets are by bot accounts. These tweets don't add any meaning to sentiment analysis. Below is a function to remove bot related tweets. We have selected bot phrases by researching online from multiple projects and combined the most common phrases.

In [333]:
def is_bot_tweet(text, bot_phrases):
    text = text.lower()
    for phrase in bot_phrases:
        if phrase in text:
            return True
    return False

bot_phrases = ['for real time prints', 'sec alert', '1 minute', 
        '#options', 'companies in our database',
        'by the same day\'s market close', 'technical alerts', 
        'new alert', 'chatroom', 'trade alert',
        'if you know trading around print', 
        'view odds for this', '15s. delayed', 'tradewithalerts']

In [334]:
filtered_amazon_tweets_df = tweet_amazon.loc[tweet_amazon.body.apply(lambda text: not is_bot_tweet(text, bot_phrases))]
filtered_tesla_tweets_df = tweet_tesla.loc[tweet_tesla.body.apply(lambda text: not is_bot_tweet(text, bot_phrases))]
filtered_microsoft_tweets_df = tweet_microsoft.loc[tweet_microsoft.body.apply(lambda text: not is_bot_tweet(text, bot_phrases))]
filtered_google_tweets_df = tweet_google.loc[tweet_google.body.apply(lambda text: not is_bot_tweet(text, bot_phrases))]
filtered_apple_tweets_df = tweet_apple.loc[tweet_apple.body.apply(lambda text: not is_bot_tweet(text, bot_phrases))]

Since there are multiple tweets about same company on a given date, we'll be combining all tweets on same date by grouping and concating them.

In [ ]:
filtered_amazon_tweets_df['body'] = filtered_amazon_tweets_df.groupby(['Date'])['body'].transform(lambda x : ' '.join(x))
filtered_tesla_tweets_df['body'] = filtered_tesla_tweets_df.groupby(['Date'])['body'].transform(lambda x : ' '.join(x))
filtered_microsoft_tweets_df['body'] = filtered_microsoft_tweets_df.groupby(['Date'])['body'].transform(lambda x : ' '.join(x))
filtered_google_tweets_df['body'] = filtered_google_tweets_df.groupby(['Date'])['body'].transform(lambda x : ' '.join(x))
filtered_apple_tweets_df['body'] = filtered_apple_tweets_df.groupby(['Date'])['body'].transform(lambda x : ' '.join(x))

In [336]:
filtered_apple_tweets_df = filtered_apple_tweets_df.drop_duplicates(subset='Date', keep="first")
filtered_tesla_tweets_df = filtered_tesla_tweets_df.drop_duplicates(subset='Date', keep="first")
filtered_microsoft_tweets_df = filtered_microsoft_tweets_df.drop_duplicates(subset='Date', keep="first")
filtered_google_tweets_df = filtered_google_tweets_df.drop_duplicates(subset='Date', keep="first")
filtered_amazon_tweets_df = filtered_amazon_tweets_df.drop_duplicates(subset='Date', keep="first")

There are lot of unwanted symbols in the tweets which will not be used in sentiment analysis. Hence, we will be removing these symbols.

In [ ]:
import re
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)   
    return input_txt    

filtered_apple_tweets_df['tidy_tweet'] = np.vectorize(remove_pattern)(filtered_apple_tweets_df['body'], "@[\w]*")
filtered_apple_tweets_df['tidy_tweet'] = filtered_apple_tweets_df['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
filtered_apple_tweets_df['tidy_tweet'] = filtered_apple_tweets_df['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

filtered_tesla_tweets_df['tidy_tweet'] = np.vectorize(remove_pattern)(filtered_tesla_tweets_df['body'], "@[\w]*")
filtered_tesla_tweets_df['tidy_tweet'] = filtered_tesla_tweets_df['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
filtered_tesla_tweets_df['tidy_tweet'] = filtered_tesla_tweets_df['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

filtered_microsoft_tweets_df['tidy_tweet'] = np.vectorize(remove_pattern)(filtered_microsoft_tweets_df['body'], "@[\w]*")
filtered_microsoft_tweets_df['tidy_tweet'] = filtered_microsoft_tweets_df['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
filtered_microsoft_tweets_df['tidy_tweet'] = filtered_microsoft_tweets_df['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

filtered_google_tweets_df['tidy_tweet'] = np.vectorize(remove_pattern)(filtered_google_tweets_df['body'], "@[\w]*")
filtered_google_tweets_df['tidy_tweet'] = filtered_google_tweets_df['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
filtered_google_tweets_df['tidy_tweet'] = filtered_google_tweets_df['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

filtered_amazon_tweets_df['tidy_tweet'] = np.vectorize(remove_pattern)(filtered_amazon_tweets_df['body'], "@[\w]*")
filtered_amazon_tweets_df['tidy_tweet'] = filtered_amazon_tweets_df['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
filtered_amazon_tweets_df['tidy_tweet'] = filtered_amazon_tweets_df['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

Saving the cleaned dataset into csv files for EDA

In [338]:
filtered_amazon_tweets_df.to_csv('dataset/filtered_amazon_tweets.csv')
filtered_tesla_tweets_df.to_csv('dataset/filtered_tesla_tweets.csv')
filtered_microsoft_tweets_df.to_csv('dataset/filtered_microsoft_tweets.csv')
filtered_google_tweets_df.to_csv('dataset/filtered_google_tweets.csv')
filtered_apple_tweets_df.to_csv('dataset/filtered_apple_tweets.csv')

Combining all filtered tweets into one dataframe and saving it into csv file 

In [339]:
filtered_tweets_df = pd.concat([filtered_amazon_tweets_df,filtered_tesla_tweets_df,filtered_microsoft_tweets_df,
                                filtered_google_tweets_df,filtered_apple_tweets_df],axis=0)
filtered_tweets_df.to_csv('dataset/filtered_tweets.csv')

Using Yahoo finance API, we will be extracting the stock prices for given duration of time

In [340]:
import pandas_datareader as web
import datetime as dt

start = dt.datetime(2019,7,1)
end = dt.datetime(2019,12,31)

amzn_stock = web.DataReader('AMZN', 'yahoo', start, end)
amzn_stock = amzn_stock.reset_index()
tesla_stock = web.DataReader('TSLA', 'yahoo', start, end)
tesla_stock = tesla_stock.reset_index()
google_stock = web.DataReader('GOOGL', 'yahoo', start, end)
google_stock = google_stock.reset_index()
microsoft_stock = web.DataReader('MSFT', 'yahoo', start, end)
microsoft_stock = microsoft_stock.reset_index()
apple_stock = web.DataReader('AAPL', 'yahoo', start, end)
apple_stock = apple_stock.reset_index()
amzn_stock.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2019-07-01,96.490997,95.733002,96.149002,96.109497,63842000,96.109497
1,2019-07-02,96.739502,95.331497,95.969002,96.715500,52918000,96.715500
2,2019-07-03,97.079498,96.525002,96.794502,96.949997,33806000,96.949997
3,2019-07-05,97.294998,96.264999,96.430000,97.145500,52568000,97.145500
4,2019-07-08,97.800003,96.412498,96.706001,97.615997,57668000,97.615997


Converting datetime column to time on stock price dataframe

In [341]:
amzn_stock['Date'] = pd.to_datetime(amzn_stock['Date']).dt.date
tesla_stock['Date'] = pd.to_datetime(tesla_stock['Date']).dt.date
google_stock['Date'] = pd.to_datetime(google_stock['Date']).dt.date
microsoft_stock['Date'] = pd.to_datetime(microsoft_stock['Date']).dt.date
apple_stock['Date'] = pd.to_datetime(apple_stock['Date']).dt.date

In [342]:
amzn_stock.to_csv('dataset/amzn_stock.csv')
tesla_stock.to_csv('dataset/tesla_stock.csv')
google_stock.to_csv('dataset/google_stock.csv')
microsoft_stock.to_csv('dataset/microsoft_stock.csv')
apple_stock.to_csv('dataset/apple_stock.csv')

Function to perform sentiment analysis on the dataframes. We have selected Vader sentiment analysis after referring to this article
https://neptune.ai/blog/sentiment-analysis-python-textblob-vs-vader-vs-flair

Compared to other libraries, vader runs quicker and gave highest accuracy. Hence, this was suitable for the size of our dataset.

In [343]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def sentiment_analysis(df):
    analyzer = SentimentIntensityAnalyzer()
    df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['tidy_tweet']]
    df['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df['tidy_tweet']]
    df['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df['tidy_tweet']]
    df['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df['tidy_tweet']]


In [ ]:
sentiment_analysis(filtered_tesla_tweets_df)
sentiment_analysis(filtered_amazon_tweets_df)
sentiment_analysis(filtered_microsoft_tweets_df)
sentiment_analysis(filtered_google_tweets_df)
sentiment_analysis(filtered_apple_tweets_df)

Adding prices columns for all the company tweets dataframes based on date

In [346]:
def prices_to_tweets(stock,tweets):
    indx=0
    for i in range (0,len(tweets)):
        for j in range (0,len(stock)):
            get_tweet_date=tweets.Date.iloc[i]
            get_stock_date=stock.Date.iloc[j]
            if(str(get_stock_date)==str(get_tweet_date)):
                tweets['Prices'].iloc[i] = float(stock.Close[j])

In [ ]:
filtered_amazon_tweets_df['Prices'] = ''
filtered_apple_tweets_df['Prices'] = ''
filtered_google_tweets_df['Prices'] = ''
filtered_microsoft_tweets_df['Prices'] = ''
filtered_tesla_tweets_df['Prices'] = ''

In [ ]:
prices_to_tweets(amzn_stock,filtered_amazon_tweets_df)
prices_to_tweets(apple_stock,filtered_apple_tweets_df)
prices_to_tweets(google_stock,filtered_google_tweets_df)
prices_to_tweets(microsoft_stock,filtered_microsoft_tweets_df)
prices_to_tweets(tesla_stock,filtered_tesla_tweets_df)

In [349]:
filtered_google_tweets_df.head()

,tweet_id,writer,Date,body,comment_num,retweet_num,like_num,ticker_symbol,tidy_tweet,compound,neg,neu,pos,Prices
3935142,1145483293372166145,TalkMarkets,2019-07-01,What Trade Earned Us 60% In June? And Which St...,0,0,0,GOOGL,What Trade Earned June Which Stocks Watch NUGT...,1.0000,0.033,0.805,0.162,55.0
3937500,1145844928422895617,Wise_Owl_Trader,2019-07-02,A wise owl free-idea; I gave this one out to m...,0,0,2,GOOG,wise free idea gave this subscribers last week...,0.9999,0.027,0.836,0.137,55.630001
3940687,1146212523571478528,SeekingAlpha,2019-07-03,$GOOG $GOOGL - Now Is The Right Time To Buy Al...,0,0,4,GOOG,GOOG GOOGL Right Time Alphabet GOOG GOOGL Righ...,0.9999,0.035,0.826,0.140,56.149502
3943547,1146574687973072896,boardtownblast,2019-07-04,https://boardtownblast.com/patch-android-july-...,0,5,1,GOOGL,https boardtownblast patch android july update...,0.9998,0.035,0.800,0.165,
3944697,1146935375941840896,sonycomeback,2019-07-05,Get ready for the #Olympics #Tokyo2020 $SNE #S...,0,1,0,GOOG,ready #Olympics #Tokyo #Sony will closer AAPL ...,0.9999,0.043,0.809,0.149,56.633499


After prices are added, some rows have blank values due to holidays, etc. We will be filling this by using mean of previous and next rows.

In [350]:
def fill_prices(df):
    mean=float(0)
    sum=float(0)
    count=float(0)
    for i in range(0,len(df)):
        if(df.Prices.iloc[i]!=""):
            sum=sum+float(df.Prices.iloc[i])
            count=count+1
    mean=float(sum/count)
    for i in range(0,len(df)):
        if(df.Prices.iloc[i]==""):
            df.Prices.iloc[i]=float(mean)

In [351]:
fill_prices(filtered_tesla_tweets_df)
fill_prices(filtered_amazon_tweets_df)
fill_prices(filtered_microsoft_tweets_df)
fill_prices(filtered_google_tweets_df)
fill_prices(filtered_apple_tweets_df)

/var/folders/st/kzrf9tws3l3173mvf544yxh40000gn/T/ipykernel_64239/2657312703.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Prices.iloc[i]=float(mean)


In [352]:
filtered_tesla_tweets_df.to_csv('dataset/Sentiment_amazon_tweets2.csv')
filtered_amazon_tweets_df.to_csv('dataset/Sentiment_tesla_tweets2.csv')
filtered_microsoft_tweets_df.to_csv('dataset/Sentiment_microsoft_tweets2.csv')
filtered_google_tweets_df.to_csv('dataset/Sentiment_google_tweets2.csv')
filtered_apple_tweets_df.to_csv('dataset/Sentiment_apple_tweets2.csv')